<a href="https://colab.research.google.com/github/Imran2205/LogicRAG/blob/master/inference/inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! git clone https://github.com/Imran2205/LogicRAG

Cloning into 'LogicRAG'...
remote: Enumerating objects: 591, done.
remote: Counting objects: 100% (591/591), done.
remote: Compressing objects: 100% (399/399), done.
remote: Total 591 (delta 169), reused 576 (delta 157), pack-reused 0 (from 0)
Receiving objects: 100% (591/591), 34.55 MiB | 18.55 MiB/s, done.
Resolving deltas: 100% (169/169), done.


In [ ]:
! gdown 1IpK8GGjgZmJ--AQaP2g-7Ax-iiJ1iqpH

Downloading...
From (original): https://drive.google.com/uc?id=1IpK8GGjgZmJ--AQaP2g-7Ax-iiJ1iqpH
From (redirected): https://drive.google.com/uc?id=1IpK8GGjgZmJ--AQaP2g-7Ax-iiJ1iqpH&confirm=t&uuid=5fda2a6c-eb0b-4c30-9514-d05c1109816d
To: /content/LogicRAG_Data.zip
100% 2.26G/2.26G [00:22<00:00, 101MB/s]


In [ ]:
! unzip LogicRAG_Data.zip

Archive:  LogicRAG_Data.zip
   creating: LogicRAG_Data/
   creating: LogicRAG_Data/model_weights/
  inflating: LogicRAG_Data/.DS_Store  
  inflating: __MACOSX/LogicRAG_Data/._.DS_Store  
   creating: LogicRAG_Data/images/
   creating: LogicRAG_Data/precomputed_knowledge_base/
   creating: LogicRAG_Data/tracker_trajectories/
   creating: LogicRAG_Data/questions/
   creating: LogicRAG_Data/model_weights/PixelFormer/
  inflating: LogicRAG_Data/model_weights/.DS_Store  
  inflating: __MACOSX/LogicRAG_Data/model_weights/._.DS_Store  
   creating: LogicRAG_Data/model_weights/Mask2Former/
   creating: LogicRAG_Data/model_weights/ColorDetector/
   creating: LogicRAG_Data/model_weights/TypeDetector/
   creating: LogicRAG_Data/model_weights/CoTracker3/
  inflating: LogicRAG_Data/images/.DS_Store  
  inflating: __MACOSX/LogicRAG_Data/images/._.DS_Store  
   creating: LogicRAG_Data/images/0003/
   creating: LogicRAG_Data/images/0004/
   creating: LogicRAG_Data/images/0005/
   creating: LogicRAG_Da

In [ ]:
! pip install anthropic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.4/243.4 kB 4.9 MB/s eta 0:00:00


In [ ]:
import os
import sys
import logging
import pandas as pd

current_directory = os.getcwd()
project_root = os.path.abspath('/content/LogicRAG')
sys.path.append(project_root)

from LogicRAG.tools.kb_framework.kb_inference.inference_in_kb import lrag_inference
from LogicRAG.tools.kb_framework.kb_inference.get_gpt4_response import get_answers_gpt4
from LogicRAG.tools.kb_framework.kb_inference.get_claude_response import get_answers_claude3

from LogicRAG.tools.kb_framework.kb_inference.utils.eval_utils import get_scores

In [ ]:
que_csv = "/content/LogicRAG_Data/questions/kitti_que.csv"
fol_trans_csv = "/content/LogicRAG_Data/questions/translated_fol_queries.csv"
kb_dir = "/content/LogicRAG_Data/precomputed_knowledge_base/kb_out_kitti"
tracker_dir = "/content/LogicRAG_Data/tracker_trajectories/track_out_kitti"
output = "./kitti_que_and_model_responses.csv"

logging.basicConfig(level=logging.INFO)
base_logger = logging.getLogger(__name__)

In [ ]:
all_answers, all_annotations = lrag_inference(
      que_csv, fol_trans_csv,
      kb_dir, tracker_dir,
      logger=base_logger
)

acc, f1, precision, recall = get_scores(all_answers, all_annotations)

print(f"Overall Accuracy: {acc:.2f}, F1: {f1:.2f}, Prec: {precision:.2f}, Rec: {recall:.2f}")

Overall Accuracy: 0.94, F1: 0.97, Prec: 0.97, Rec: 0.97


In [ ]:
df = pd.read_csv(que_csv)
df['LogicRAG'] = all_answers
df.to_csv(output, index=False)

In [ ]:
GPT_MODEL_NAME = "gpt-4o"  # gpt-4o-2024-08-06
CLAUDE_MODEL_NAME = "claude-3-5-sonnet-20240620"

os.environ["ANTHROPIC_API_KEY"] = "your_anthropic_api_key"
os.environ["OPENAI_API"] = "your_openai_api_key"